# Computations
to get correct values to compare with pytest assert

In [1]:
import numpy as np
import os
import pickle

In [2]:
import sys
sys.path.append('../')  # since MDRefine is in a different folder

### compute_new_weights, compute_D_KL and l2_regularization

In [ ]:
from MDRefine import compute_new_weights, compute_D_KL, l2_regularization

In [7]:
w0 = np.array([0.5, 0.5])
correction = np.array([0, 1])

w, logZ = compute_new_weights(w0, correction)

print(w, logZ)

[0.73105858 0.26894142] -0.3798854930417225


In [8]:
compute_D_KL(weights_P=w, correction_ff=1/2*correction, temperature=2, logZ_P=logZ)

DeviceArray(0.31265014, dtype=float64)

In [10]:
pars = np.array([1.2, 1.5])

l2_regularization(pars)

(DeviceArray(3.69, dtype=float64), array([2.4, 3. ]))

### copy data and reduce them (less frames and observables)

just to test

In [58]:
!rm DATA_test/CAAU/pdb_structure.pdb
!rm DATA_test/CAAU/ff_terms
!rm DATA_test/CAAU/ff_terms_chi
!rm DATA_test/CAAU/replica_temp

In [59]:
!rm DATA_test/fm_coeffs_Thorben_ref

In [57]:
for s in ['CAAU']:  # 'AAAA', 'CAAU']:
    for s2 in ['forward_qs']:
        path = 'DATA_test/%s/%s/' % (s, s2)
        file_names = os.listdir('DATA_test/' + s + '/' + s2 + '/')
        for s3 in file_names:
            vec = np.load(path + s3)[::10000, :2]
            np.save(path + s3, vec)

    for s2 in ['g_exp', 'names']:
        path = 'DATA_test/%s/%s/' % (s, s2)
        file_names = os.listdir('DATA_test/' + s + '/' + s2 + '/')
        for s3 in file_names:
            vec = np.load(path + s3)[:2]
            np.save(path + s3, vec)

    for s2 in ['g']:
        path = 'DATA_test/%s/observables/' % s
        file_names = ['NOEs', 'uNOEs']
        for s3 in file_names:
            vec = np.load(path + s3 + '.npy')[::10000, :2]
            np.save(path + s3, vec)

    vec = np.load('DATA_test/%s/ff_terms.npy' % s)[::10000]
    np.save('DATA_test/%s/ff_terms.npy' % s, vec)

    vec = np.load('DATA_test/%s/replica_temp.npy' % s)[::10000]
    np.save('DATA_test/%s/replica_temp.npy' % s, vec)



In [63]:
for s in ['A1_AD', 'A1_MD', 'A1_AS', 'A1_MS']:
    vec = np.load('DATA_test/%s/weights.npy' % s)[::1000]
    np.save('DATA_test/%s/weights.npy' % s, vec)

for s in ['A1_MD', 'A1_MS']:
    vec = np.load('DATA_test/%s/ff_terms.npy' % s)[::1000]
    np.save('DATA_test/%s/ff_terms.npy' % s, vec)

#### load data and save as pickle

In [11]:
from MDRefine import load_data

In [35]:
infos = {}
infos['global'] = {'path_directory': 'DATA_test', 'system_names': ['AAAA', 'CAAU']}

for name in infos['global']['system_names']:
    infos[name] = {}
    infos[name]['g_exp'] = ['NOEs', ('uNOEs','<')]
    infos[name]['obs'] = ['NOEs', 'uNOEs']

infos['global']['temperature'] = 1 # namely, energies are in unit of k_B T (default value)

data = load_data(infos, stride=2)


loading data from directory...
loading  AAAA
loading  CAAU
done


In [36]:
with open('DATA_test/data_stride2.pkl', 'wb') as f:
    pickle.dump(vars(data), f)

### test load_data

compare output of load_data with the pre-loaded data object, which is stored as pickle

In [3]:
from MDRefine import load_data

In [4]:
infos = {}

# Firstly, define global properties, valid for all the systems:

infos['global'] = {'path_directory': 'DATA_test', 'system_names': ['AAAA', 'CAAU']}

# Then, define properties which are specific of each system, like experimental data and observables

for name in infos['global']['system_names']:
    infos[name] = {}
    
    # experimental observables (average and uncertainty), corresponding to 'file_name'.npy in DATA/system_name/g_exp/
    # uNOEs values are upper bounds, so specify '<' with ('uNOEs','<')
    infos[name]['g_exp'] = ['NOEs', ('uNOEs','<')]
    
    # observables from MD simulations, corresponding to 'file_name'.npy in DATA/system_name/observables/
    # they must correspond also to items of infos[name]['g_exp']
    infos[name]['obs'] = ['NOEs', 'uNOEs']

# If some properties are the same for all the systems, you can store them just once in infos['global']

infos['global']['temperature'] = 1 # namely, energies are in unit of k_B T (default value)
# (in this case, you could do this also for 'g_exp' and 'obs')

In [5]:
data = load_data(infos)

loading data from directory...
loading  AAAA
loading  CAAU
done


In [6]:
vars(data.sys['AAAA'])

{'temperature': 1,
 'gexp': {'NOEs': array([[7.74312661e-04, 4.56174227e-04],
         [8.17622013e-05, 7.09253186e-05]]),
  'uNOEs': array([[1.58193730e-04, 2.39877632e-05],
         [7.22476158e-05, 9.51458553e-06]])},
 'names': {'NOEs': array([["A1-H1'", 'A1-H8'],
         ["A1-H1'", 'A2-H8']], dtype='<U7'),
  'uNOEs': array([["A1-H1'", "A2-H4'"],
         ["A1-H1'", "A3-H1'"]], dtype='<U7')},
 'ref': {'NOEs': '=', 'uNOEs': '<'},
 'g': {'NOEs': memmap([[3.78438242e-04, 6.64434046e-05],
          [8.24084855e-04, 5.37090818e-05],
          [4.24279075e-04, 8.45648974e-05],
          [5.57313382e-04, 6.57341152e-05],
          [3.65299667e-04, 3.69220197e-06],
          [2.89047486e-03, 1.00485659e-05],
          [4.10240405e-04, 8.74714679e-05],
          [3.12228408e-03, 1.53735527e-05],
          [4.78437869e-03, 1.16904630e-05],
          [2.48920987e-04, 1.28376223e-06],
          [2.85325595e-03, 1.08458007e-05],
          [7.43237499e-04, 2.90249372e-05],
          [2.33285697e

In [6]:
with open('data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [55]:
vars(data._global_).keys()

dict_keys(['system_names'])

In [66]:
vars(loaded_data['_global_'])

{'system_names': ['AAAA', 'CAAU']}

In [67]:
dir(loaded_data['_global_'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'system_names',
 'tot_n_experiments']

In [70]:
data._global_.system_names

['AAAA', 'CAAU']

In [7]:
data._global_.tot_n_experiments(data)

8

In [9]:
class my_data():
    def __init__(self):
        self.sys = {}

my_loaded_data = my_data()
my_loaded_data.sys['AAAA'] = loaded_data['sys']['AAAA']
my_loaded_data.sys['CAAU'] = loaded_data['sys']['CAAU']


In [10]:
loaded_data['_global_'].tot_n_experiments(my_loaded_data)

8

In [74]:
# assert list(vars(data._global_).keys()) == list(vars(loaded_data['_global_']).keys())
assert dir(data._global_) == dir(loaded_data['_global_'])

assert data._global_.system_names == loaded_data['_global_'].system_names
assert data._global_.tot_n_experiments(data) == loaded_data['_global_'].tot_n_experiments(loaded_data)

AttributeError: 'dict' object has no attribute 'sys'

In [77]:
# loaded_data['_global_'].tot_n_experiments(loaded_data)

loaded_data

{'_global_': <MDRefine.data_loading.data_global_class at 0x7f3eb86abcc0>,
 'sys': {'AAAA': <MDRefine.data_loading.data_class at 0x7f3eb86abc18>,
  'CAAU': <MDRefine.data_loading.data_class at 0x7f3eb86ab940>}}

In [59]:
data._global_.__dir__()

['system_names',
 '__module__',
 '__doc__',
 '__init__',
 'tot_n_experiments',
 '__dict__',
 '__weakref__',
 '__repr__',
 '__hash__',
 '__str__',
 '__getattribute__',
 '__setattr__',
 '__delattr__',
 '__lt__',
 '__le__',
 '__eq__',
 '__ne__',
 '__gt__',
 '__ge__',
 '__new__',
 '__reduce_ex__',
 '__reduce__',
 '__subclasshook__',
 '__init_subclass__',
 '__format__',
 '__sizeof__',
 '__dir__',
 '__class__']

In [12]:
assert vars(data.sys['AAAA']) == vars(loaded_dict['sys']['AAAA'])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

#### assert does not work
because we cannot compare two dictionaries containing numpy arrays

In [15]:
my_dict1 = {'a': np.array([1.5, 2.5]), 'b': np.array([1.2, 2.4, 3.6])}
my_dict2 = {'a': np.array([1.5, 2.5]), 'b': np.array([1.2, 2.4, 3.6])}

assert (my_dict1 == my_dict2).all()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

this works:

In [16]:
assert (my_dict1['a'] == my_dict2['a']).all()

#### so, let's proceed for extended

In [30]:
vars(data.sys['AAAA'])

{'temperature': 1,
 'gexp': {'NOEs': array([[7.74312661e-04, 4.56174227e-04],
         [8.17622013e-05, 7.09253186e-05]]),
  'uNOEs': array([[1.58193730e-04, 2.39877632e-05],
         [7.22476158e-05, 9.51458553e-06]])},
 'names': {'NOEs': array([["A1-H1'", 'A1-H8'],
         ["A1-H1'", 'A2-H8']], dtype='<U7'),
  'uNOEs': array([["A1-H1'", "A2-H4'"],
         ["A1-H1'", "A3-H1'"]], dtype='<U7')},
 'ref': {'NOEs': '=', 'uNOEs': '<'},
 'g': {'NOEs': memmap([[3.78438242e-04, 6.64434046e-05],
          [8.24084855e-04, 5.37090818e-05],
          [4.24279075e-04, 8.45648974e-05],
          [5.57313382e-04, 6.57341152e-05],
          [3.65299667e-04, 3.69220197e-06],
          [2.89047486e-03, 1.00485659e-05],
          [4.10240405e-04, 8.74714679e-05],
          [3.12228408e-03, 1.53735527e-05],
          [4.78437869e-03, 1.16904630e-05],
          [2.48920987e-04, 1.28376223e-06],
          [2.85325595e-03, 1.08458007e-05],
          [7.43237499e-04, 2.90249372e-05],
          [2.33285697e

In [28]:
vars(data.sys['AAAA']).keys()


dict_keys(['temperature', 'gexp', 'names', 'ref', 'g', 'weights', 'n_experiments', 'n_frames'])

In [35]:
my_dict1['n_experiments']

{'NOEs': 2, 'uNOEs': 2}

In [33]:
s = 'AAAA'

my_dict1 = vars(data.sys[s])
my_dict2 = vars(loaded_dict['sys'][s])

for k in my_dict1.keys():

    if k in ['temperature', 'ref', 'n_experiments', 'n_frames']:
        assert my_dict1[k] == my_dict2[k]

    elif k in ['gexp', 'names', 'g']:
        for k2 in data.sys[s].gexp.keys():
            assert (my_dict1[k][k2] == my_dict2[k][k2]).all()

    elif k in ['weights']:
        assert (my_dict1[k] == my_dict2[k]).all()



In [39]:
self.assertlist(my_dict1.keys()) == list(my_dict2.keys())

In [41]:
my_dict1

{'temperature': 1,
 'gexp': {'NOEs': array([[7.74312661e-04, 4.56174227e-04],
         [8.17622013e-05, 7.09253186e-05]]),
  'uNOEs': array([[1.58193730e-04, 2.39877632e-05],
         [7.22476158e-05, 9.51458553e-06]])},
 'names': {'NOEs': array([["A1-H1'", 'A1-H8'],
         ["A1-H1'", 'A2-H8']], dtype='<U7'),
  'uNOEs': array([["A1-H1'", "A2-H4'"],
         ["A1-H1'", "A3-H1'"]], dtype='<U7')},
 'ref': {'NOEs': '=', 'uNOEs': '<'},
 'g': {'NOEs': memmap([[3.78438242e-04, 6.64434046e-05],
          [8.24084855e-04, 5.37090818e-05],
          [4.24279075e-04, 8.45648974e-05],
          [5.57313382e-04, 6.57341152e-05],
          [3.65299667e-04, 3.69220197e-06],
          [2.89047486e-03, 1.00485659e-05],
          [4.10240405e-04, 8.74714679e-05],
          [3.12228408e-03, 1.53735527e-05],
          [4.78437869e-03, 1.16904630e-05],
          [2.48920987e-04, 1.28376223e-06],
          [2.85325595e-03, 1.08458007e-05],
          [7.43237499e-04, 2.90249372e-05],
          [2.33285697e

### include also forward model and force-field correction

save complete pickle with stride=2 as data_complete_stride2.pkl

In [43]:
infos = {'global': {
    'path_directory': 'DATA_test',
    'system_names': ['AAAA', 'CAAU'],
    'g_exp': ['backbone1_gamma_3J', 'backbone2_beta_epsilon_3J', 'sugar_3J', 'NOEs' , ('uNOEs', '<')],
    'forward_qs': ['backbone1_gamma', 'backbone2_beta_epsilon','sugar'],
    'obs': ['NOEs', 'uNOEs'],
    'forward_coeffs': 'original_fm_coeffs'}}

stride = 2

In [44]:
def forward_model_fun(fm_coeffs, forward_qs, selected_obs=None):

    # 1. compute the cosine (which is the quantity you need in the forward model;
    # you could do this just once before loading data)
    forward_qs_cos = {}

    for type_name in forward_qs.keys():
        forward_qs_cos[type_name] = jnp.cos(forward_qs[type_name])

    # if you have selected_obs, compute only the corresponding observables
    if selected_obs is not None:
        for type_name in forward_qs.keys():
            forward_qs_cos[type_name] = forward_qs_cos[type_name][:,selected_obs[type_name+'_3J']]

    # 2. compute observables (forward_qs_out) through forward model
    forward_qs_out = {
        'backbone1_gamma_3J': fm_coeffs[0]*forward_qs_cos['backbone1_gamma']**2 + fm_coeffs[1]*forward_qs_cos['backbone1_gamma'] + fm_coeffs[2],
        'backbone2_beta_epsilon_3J': fm_coeffs[3]*forward_qs_cos['backbone2_beta_epsilon']**2 + fm_coeffs[4]*forward_qs_cos['backbone2_beta_epsilon'] + fm_coeffs[5],
        'sugar_3J': fm_coeffs[6]*forward_qs_cos['sugar']**2 + fm_coeffs[7]*forward_qs_cos['sugar'] + fm_coeffs[8] }

    return forward_qs_out

In [47]:
infos['global']['forward_model'] = forward_model_fun

In [31]:
import jax.numpy as jnp

In [45]:
infos['global']['names_ff_pars'] = ['sin alpha', 'cos alpha']

def ff_correction(pars, f):
    out = jnp.matmul(pars, (f[:, [0, 6]] + f[:, [1, 7]] + f[:, [2, 8]]).T)
    return out

def ff_correction_hexamers(pars, f):
    out = jnp.matmul(pars, (f[:, [0, 10]] + f[:, [1, 11]] + f[:, [2, 12]] + f[:, [3, 13]] + f[:, [4, 14]]).T)
    return out

infos['global']['ff_correction'] = ff_correction

In [48]:
data = load_data(infos, stride=stride)

loading data from directory...
loading  AAAA
loading  CAAU
done


In [77]:
del data.sys['AAAA'].forward_model
del data.sys['CAAU'].forward_model

In [75]:
del data.sys['AAAA'].ff_correction
del data.sys['CAAU'].ff_correction

In [57]:
def my_forward_model(a, b, c=None):
    try:
        out = infos['global']['forward_model'](a, b, c)
    except:
        assert c is None, 'you have selected_obs but the forward model is not suitably defined!'
        out = infos['global']['forward_model'](a, b)
    return out

data.sys['AAAA'].forward_model = my_forward_model  # info['forward_model']
data.sys['CAAU'].forward_model = my_forward_model  # info['forward_model']

In [72]:
with open('DATA_test/data_complete_stride2.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [73]:
dir(loaded_data['_global_'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'forward_coeffs_0',
 'names_ff_pars',
 'system_names',
 'tot_n_experiments']

In [69]:
list(loaded_data['_global_'].forward_coeffs_0)
list(loaded_data['_global_'].forward_coeffs_0.keys())

['A_gamma',
 'B_gamma',
 'C_gamma',
 'A_beta',
 'B_beta',
 'C_beta',
 'A_sugar',
 'B_sugar',
 'C_sugar']

In [67]:
assert (list(loaded_data['_global_'].forward_coeffs_0) == list(data._global_.forward_coeffs_0)).all()
assert list(loaded_data['_global_'].forward_coeffs_0.keys()) == list(data._global_.forward_coeffs_0.keys())

In [70]:
loaded_data['_global_'].names_ff_pars

['sin alpha', 'cos alpha']

### test on data for alchemical calculations

In [81]:
infos = {'global': {'temperature': 2.476, 'path_directory': 'DATA_test'}}

cycle_names = ['A1']

names = {}
for name in cycle_names:
    names[name] = []
    for string in ['AS','AD','MS','MD']:
        names[name].append((name + '_' + string))

infos['global']['cycle_names'] = names
infos['global']['system_names'] = [s2 for s in list(names.values()) for s2 in s]

# force-field correction terms

n_charges = 5

infos['global']['names_ff_pars'] = ['DQ %i' % (i+1) for i in range(n_charges)] + ['cos eta']

columns = []
for i in range(n_charges):
    columns.append('DQ %i' % (i+1))
    columns.append('DQ %i%i' % (i+1,i+1))
for i in range(n_charges):
    for j in range(i+1,n_charges):
        columns.append('DQ %i%i' % (i+1,j+1))
columns.append('cos eta')

# only methylated (M) systems have a force-field correction

for name in infos['global']['system_names']: infos[name] = {}

for name in infos['global']['cycle_names'].keys():
    for s in ['D','S']:
        infos[name + '_M' + s]['ff_terms'] = columns

In [82]:
names_charges = ['N6', 'H61', 'N1', 'C10', 'H101/2/3']

In [83]:
def ff_correction(phi, ff_terms):

    n_charges = 5

    phi_vector = []
    for i in range(n_charges):
        phi_vector.extend([phi[i], phi[i]**2])
    for i in range(n_charges):
        for j in range(i+1,n_charges):
            phi_vector.append(phi[i]*phi[j])
    phi_vector.append(-phi[-1])
    phi_vector = jnp.array(phi_vector)

    correction = jnp.matmul(ff_terms, phi_vector)

    return correction

In [84]:
for k in infos['global']['system_names']:
    if k[-2] == 'M': 
        infos[k]['ff_correction'] = ff_correction

In [87]:
data = load_data(infos, stride=2)

loading data from directory...
loading  A1_AS
loading  A1_AD
loading  A1_MS
loading  A1_MD
done


In [93]:
del data.sys['A1_MS'].ff_correction
del data.sys['A1_MD'].ff_correction

In [89]:
with open('DATA_test/data_alchemical_stride2.pkl', 'rb') as f:
    loaded_data = pickle.load(f)

In [90]:
loaded_data

{'_global_': <MDRefine.data_loading.data_global_class at 0x7f32c06b0438>,
 'sys': {'A1_AS': <MDRefine.data_loading.data_class at 0x7f32c057f2b0>,
  'A1_AD': <MDRefine.data_loading.data_class at 0x7f32c057f1d0>,
  'A1_MS': <MDRefine.data_loading.data_class at 0x7f32c057f0b8>,
  'A1_MD': <MDRefine.data_loading.data_class at 0x7f32c057f828>},
 'cycle': {'A1': <MDRefine.data_loading.data_cycle_class at 0x7f32c057f8d0>}}

In [105]:
vars(data.cycle['A1'])

{'gexp_DDG': [6.3, 0.5], 'temperature': 2.476}

In [104]:
data.cycle

{'A1': <MDRefine.data_loading.data_cycle_class at 0x7f32c0554390>}

In [103]:
loaded_data['cycle']['A1'].temperature

loaded_data['cycle']['A1'].gexp_DDG

[6.3, 0.5]